# Recurrent Neural Network

In [2]:
import torch
from torch import nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [4]:
# configure hyperparameters
EPOCH = 1                # epoch: train the training data n times; 1 epoch is used here to save training time
BATCH_SIZE = 64
TIME_STEP = 28           # time_step: rnn time step / image height
INPUT_SIZE = 28          # input_size: rnn input size / image width
LR = 0.01                # lr: learning rate
DOWNLOAD_MNIST = False   # download_mnist: set it to True if the mnist data has not been downloaded

In [5]:
# get mnist digital dataset
train_data = dsets.MNIST(
    root='data/mnist/',
    train=True,                         # set up the training data
    transform=transforms.ToTensor(),    # convert a PIL.Image or numpy.ndarray to torch.FloatTensor of shape (C x H x W) and normalize it in the range [0.0, 1.0]
    download=DOWNLOAD_MNIST,            # download the mnist data if non-exist
)

In [6]:
# configure training data loader for easy mini-batch return
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)

In [8]:
# convert test data into variables; select 2000 samples to accelerate testing
test_data = dsets.MNIST(root='data/mnist/', train=False, transform=transforms.ToTensor())
test_x = test_data.test_data.type(torch.FloatTensor)[:2000]/255. # shape (2000, 28, 28) value in range(0,1)
test_y = test_data.test_labels.numpy()[:2000] # covert to numpy array

In [9]:
# build the rnn architecture
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()

        self.rnn = nn.LSTM(         # not use nn.RNN() here, as it hardly learns the parameters
            input_size=INPUT_SIZE,
            hidden_size=64,         # hidden_size: rnn hidden unit dimensionality
            num_layers=1,           # num_layers: number of rnn layers
            batch_first=True,       # batch_first: input and output will has batch size as 1st dimension. e.g. (batch, time_step, input_size)
        )

        self.out = nn.Linear(64, 10)

    def forward(self, x):
        # x shape (batch, time_step, input_size)
        # r_out shape (batch, time_step, output_size)
        # h_n shape (n_layers, batch, hidden_size)
        # h_c shape (n_layers, batch, hidden_size)
        r_out, (h_n, h_c) = self.rnn(x, None)   # None means zero initial hidden state

        # choose r_out at the last time step
        out = self.out(r_out[:, -1, :])
        return out

In [10]:
rnn = RNN()
print(rnn)

RNN(
  (rnn): LSTM(28, 64, batch_first=True)
  (out): Linear(in_features=64, out_features=10, bias=True)
)


In [11]:
optimizer = torch.optim.Adam(rnn.parameters(), lr=LR)   
loss_func = nn.CrossEntropyLoss()                       

In [12]:
# training and testing
for epoch in range(EPOCH):
    for step, (b_x, b_y) in enumerate(train_loader):    # feed batch data
        b_x = b_x.view(-1, 28, 28)                      # reshape x to (batch, time_step, input_size)

        output = rnn(b_x)                               # rnn output
        loss = loss_func(output, b_y)                   # cross entropy loss
        optimizer.zero_grad()                           # clear gradients for this training step
        loss.backward()                                 # backpropagation, compute gradients
        optimizer.step()                                # apply gradients

        if step % 50 == 0:
            test_output = rnn(test_x)                   # (samples, time_step, input_size)
            pred_y = torch.max(test_output, 1)[1].data.numpy()
            accuracy = float((pred_y == test_y).astype(int).sum()) / float(test_y.size)
            print('Epoch: ', epoch, '| train loss: %.4f' % loss.data.numpy(), '| test accuracy: %.2f' % accuracy)

Epoch:  0 | train loss: 2.3006 | test accuracy: 0.10
Epoch:  0 | train loss: 1.2295 | test accuracy: 0.56
Epoch:  0 | train loss: 0.6107 | test accuracy: 0.75
Epoch:  0 | train loss: 0.6255 | test accuracy: 0.82
Epoch:  0 | train loss: 0.4475 | test accuracy: 0.86
Epoch:  0 | train loss: 0.3409 | test accuracy: 0.88
Epoch:  0 | train loss: 0.2818 | test accuracy: 0.91
Epoch:  0 | train loss: 0.2736 | test accuracy: 0.90
Epoch:  0 | train loss: 0.3692 | test accuracy: 0.93
Epoch:  0 | train loss: 0.0663 | test accuracy: 0.94
Epoch:  0 | train loss: 0.1243 | test accuracy: 0.93
Epoch:  0 | train loss: 0.1971 | test accuracy: 0.94
Epoch:  0 | train loss: 0.0638 | test accuracy: 0.94
Epoch:  0 | train loss: 0.1598 | test accuracy: 0.95
Epoch:  0 | train loss: 0.1742 | test accuracy: 0.94
Epoch:  0 | train loss: 0.0557 | test accuracy: 0.95
Epoch:  0 | train loss: 0.0577 | test accuracy: 0.95
Epoch:  0 | train loss: 0.2175 | test accuracy: 0.96
Epoch:  0 | train loss: 0.1038 | test accuracy

In [13]:
# print 10 predictions from test data
test_output = rnn(test_x[:10].view(-1, 28, 28))
pred_y = torch.max(test_output, 1)[1].data.numpy()
print(pred_y, 'prediction number')
print(test_y[:10], 'real number')

[7 2 1 0 4 1 4 9 5 9] prediction number
[7 2 1 0 4 1 4 9 5 9] real number
